In [ ]:
!pip install matplotlib numpy qutip ipywidgets ipympl IPython

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import Bloch, about, basis, mesolve, sigmam, sigmax, sigmay, sigmaz
from qutip.ipynbtools import plot_animation
from IPython.display import HTML, display
import matplotlib.animation as animation

%matplotlib inline

In [ ]:
def qubit_integrate_random(psi0, tlist):
    """
    Integrate qubit with time-varying random parameters for more chaotic motion
    """
    sx = sigmax()
    sy = sigmay()
    sz = sigmaz()
    sm = sigmam()

    # Create time-dependent Hamiltonian for more random motion
    # We'll use a combination of rotating fields
    def H_coeff_x(t, args):
        return np.cos(2*np.pi*t/3) + 0.5*np.sin(5*np.pi*t/7)

    def H_coeff_y(t, args):
        return np.sin(2*np.pi*t/5) + 0.3*np.cos(7*np.pi*t/11)

    def H_coeff_z(t, args):
        return 0.8*np.cos(2*np.pi*t/4) + 0.4*np.sin(3*np.pi*t/8)

    H = [
        [sx, H_coeff_x],
        [sy, H_coeff_y],
        [sz, H_coeff_z]
    ]

    # Weaker collapse operators for longer-lasting motion
    c_op_list = []
    gamma1 = 0.03  # Reduced relaxation rate
    gamma2 = 0.015  # Reduced dephasing rate
    n_th = 0.3

    rate = gamma1 * (n_th + 1)
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * sm)
    rate = gamma1 * n_th
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * sm.dag())
    rate = gamma2
    if rate > 0.0:
        c_op_list.append(np.sqrt(rate) * sz)

    # evolve and calculate expectation values
    output = mesolve(H, psi0, tlist, c_op_list, [sx, sy, sz])
    return output


In [ ]:
# Initial state - superposition
a = 0.7
psi0 = (a * basis(2, 0) + (1 - a) * basis(2, 1)) / (np.sqrt(a**2 + (1 - a)**2))

# Longer time evolution for more interesting dynamics
tlist = np.linspace(0, 30, 800)  # 30 time units, 800 frames

# Run simulation
result = qubit_integrate_random(psi0, tlist)

/usr/local/lib/python3.12/dist-packages/qutip/solver/solver_base.py:598: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


In [ ]:
def calculate_state_amplitudes(result, n):
    """Calculate alpha and beta coefficients from expectation values"""
    # State vector reconstruction from Bloch vector
    x, y, z = result.expect[0][n], result.expect[1][n], result.expect[2][n]

    # For a pure state on the Bloch sphere:
    # |ψ⟩ = cos(θ/2)|0⟩ + e^(iφ)sin(θ/2)|1⟩
    # where θ = arccos(z), φ = arctan2(y, x)

    r = np.sqrt(x**2 + y**2 + z**2)
    if r > 0:
        theta = np.arccos(z / r)
        phi = np.arctan2(y, x)
    else:
        theta = 0
        phi = 0

    # Calculate alpha and beta
    alpha = np.cos(theta / 2)
    beta = np.exp(1j * phi) * np.sin(theta / 2)

    return alpha, beta


In [ ]:
def plot_setup(result):
    """Setup the figure with dark theme"""
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(12, 10))
    fig.patch.set_facecolor('black')

    axes = fig.add_subplot(111, projection="3d")
    axes.set_facecolor('black')

    # Remove grid
    axes.grid(False)

    # Remove pane fills
    axes.xaxis.pane.fill = False
    axes.yaxis.pane.fill = False
    axes.zaxis.pane.fill = False

    # Make axis lines more clear with cyan neon
    axes.xaxis.line.set_color('#00ffff')
    axes.yaxis.line.set_color('#00ffff')
    axes.zaxis.line.set_color('#00ffff')
    axes.xaxis.line.set_linewidth(2)
    axes.yaxis.line.set_linewidth(2)
    axes.zaxis.line.set_linewidth(2)

    return fig, axes

In [ ]:
sphere = None
trail_points_x = []
trail_points_y = []
trail_points_z = []

def plot_result(result, n, fig=None, axes=None, azim=None):
    global sphere, trail_points_x, trail_points_y, trail_points_z

    if fig is None or axes is None:
        fig, axes = plot_setup(result)

    # Create Bloch sphere only once
    if not sphere:
        sphere = Bloch(axes=axes)

        sphere.sphere_color = '#1a1a2e'
        sphere.sphere_alpha = 0.2
        sphere.frame_alpha = 0.3
        sphere.frame_width = 2
        sphere.font_color = '#e0e0e0'
        sphere.font_size = 14

        sphere.vector_color = []
        sphere.point_color = []

    sphere.clear()

    current_x = result.expect[0][n]
    current_y = result.expect[1][n]
    current_z = result.expect[2][n]

    sphere.make_sphere()

    # Use provided azimuth or auto-rotate
    if azim is not None:
        axes.view_init(elev=20, azim=azim)
    else:
        axes.view_init(elev=20, azim=n * 0.8)

    # Trail effect - keep last 50 points with neon cyan fading glow
    trail_length = 50

    # Add every point for coherent trail
    trail_points_x.append(current_x)
    trail_points_y.append(current_y)
    trail_points_z.append(current_z)

    # Keep only last trail_length points
    if len(trail_points_x) > trail_length:
        trail_points_x.pop(0)
        trail_points_y.pop(0)
        trail_points_z.pop(0)

    # Plot trail with neon cyan fading glow effect - multiple passes for glow
    if len(trail_points_x) > 1:
        # Create glow effect with multiple layers
        for glow_pass in range(3):
            glow_width = 5 - glow_pass * 1.5
            glow_alpha_mult = 0.5 - glow_pass * 0.15

            for i in range(len(trail_points_x) - 1):
                # Fade from back to front
                alpha = (i + 1) / len(trail_points_x) * glow_alpha_mult
                axes.plot([trail_points_x[i], trail_points_x[i+1]],
                         [trail_points_y[i], trail_points_y[i+1]],
                         [trail_points_z[i], trail_points_z[i+1]],
                         '-', color='#00ffff', alpha=alpha,
                         linewidth=glow_width)

    # State vector - White arrow
    axes.plot([0, current_x],
              [0, current_y],
              [0, current_z],
              color='white',
              linewidth=5)

    # Bright cyan tip point at arrow end
    axes.scatter([current_x],
                 [current_y],
                 [current_z],
                 color='#00ffff',
                 s=100,
                 alpha=0.8,
                 edgecolors='white',
                 linewidths=2,
                 depthshade=False)

    # Calculate theta and phi from Bloch vector
    r = np.sqrt(current_x**2 + current_y**2 + current_z**2)
    if r > 0:
        theta = np.arccos(current_z / r)
        phi = np.arctan2(current_y, current_x)
    else:
        theta = 0
        phi = 0

    # Convert to degrees for display
    theta_deg = np.degrees(theta)
    phi_deg = np.degrees(phi)

    # Quantum state legend - positioned at top right, no box
    state_text = f'|ψ⟩ = cos(θ/2)|0⟩ + e^(iφ)sin(θ/2)|1⟩\nθ = {theta_deg:.1f}°\nφ = {phi_deg:.1f}°'

    axes.text2D(0.98, 0.98, state_text,
                transform=axes.transAxes,
                fontsize=14,
                color='white',
                horizontalalignment='right',
                verticalalignment='top')

    return axes.artists

In [ ]:
# Create and save animation
fig, axes = plot_setup(result)

def animate(n):
    # Rotate the view slowly (360 degrees over the full animation)
    azim = (n / len(tlist)) * 360
    return plot_result(result, n, fig, axes, azim)

print("Creating animation... This may take a minute.")

anim = animation.FuncAnimation(
    fig,
    animate,
    frames=len(tlist),
    interval=40,
    blit=False,
    repeat=True
)

# Save as GIF
print("Saving animation as GIF...")
anim.save('enhanced_qubit_bloch_sphere.gif', writer='pillow', fps=25, dpi=100)
plt.close()

print("Animation saved as 'enhanced_qubit_bloch_sphere.gif'")
print(f"Total frames: {len(tlist)}")
print(f"Duration: {tlist[-1]:.1f} time units")


Creating animation... This may take a minute.
Saving animation as GIF...
Animation saved as 'enhanced_qubit_bloch_sphere.gif'
Total frames: 800
Duration: 30.0 time units


In [ ]:
# Display the GIF in the notebook
display(HTML('<img src="enhanced_qubit_bloch_sphere.gif">'))